In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 

from vehicle_model_DDQN3_2 import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_e-4wd_Battery.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_id_75_110_Westinghouse.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 1)


In [3]:
# STATE_SIZE = env.calculation_comp["state_size"]
STATE_SIZE = 4
ACTION_SIZE = env.calculation_comp["action_size"] 
LEARNING_RATE = 0.00025 

TOTAL_EPISODES = 200
MAX_STEPS = 50000 

GAMMA = 0.95 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
TAU = 0.001 
DELAY_TRAINING = 3000 
EPSILON_MIN_ITER = 5000

In [4]:
primary_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])
target_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])

primary_network.compile(
    loss="mse", 
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
)

# for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
#     t.assign(p)

In [5]:
def update_network(primary_network, target_network): 
    for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
        t.assign(t * (1 - TAU) + p * TAU)

In [6]:
class Memory: 
    def __init__(self, max_memory): 
        self.max_memory = max_memory 
        self._samples = [] 
        
    def add_sample(self, sample): 
        self._samples.append(sample)
        if len(self._samples) > self.max_memory: 
            self._samples.pop(0)
        
    def sample(self, no_samples): 
        if no_samples > len(self._samples): 
            return random.sample(self._samples, len(self._samples))
        else: 
            return random.sample(self._samples, no_samples)
    
    @property
    def num_samples(self):
        return len(self._samples)
    

# memory = Memory(10000)

In [7]:
def choose_action(state, primary_network, eps): 
    if random.random() < eps: 
        return random.randint(0, ACTION_SIZE - 1)
    else: 
        return np.argmax(primary_network(np.array(state).reshape(1, -1))) 

In [8]:
def train(primary_network, target_network, memory): 
    batch = memory.sample(BATCH_SIZE)
    states = np.array([val[0] for val in batch]) 
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([np.zeros(STATE_SIZE) if val[3] is None else val[3]  
                            for val in batch])
    
    prim_qt = primary_network(states)
    prim_qtp1 = primary_network(next_states)
    target_q = prim_qt.numpy() 
    updates = rewards 
    valid_idxs = next_states.sum(axis=1) != 0 
    batch_idxs = np.arange(BATCH_SIZE)
    prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
    q_from_target = target_network(next_states)
    updates[valid_idxs] += GAMMA * q_from_target.numpy()[batch_idxs[valid_idxs], 
                                                        prim_action_tp1[valid_idxs]]
    
    target_q[batch_idxs, actions] = updates 
    loss = primary_network.train_on_batch(states, target_q)
    return loss 
    
    
    

In [9]:
def initialization_with_rewardFactor(reward_factor):
    env = Environment(cell_model, drving_cycle, battery_path, motor_path, reward_factor)
    
    memory = Memory(10000)
    
    primary_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(),  
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    target_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    primary_network.compile(
        loss="mse", 
        optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
    )
    return env, memory, primary_network, target_network 
    

In [10]:
print("environment version: {}".format(env.version)) 

 
reward_factors = [10]
results_dict = {} 

for reward_factor in reward_factors: 
    eps = MAX_EPSILON 
    steps = 0
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    env, memory, primary_network, target_network = initialization_with_rewardFactor(reward_factor)
    for episode in range(TOTAL_EPISODES): 
        state = env.reset() 
        avg_loss = 0 
        total_reward = 0
        cnt = 1 

        while True:
            action = choose_action(state, primary_network, eps)
            next_state, reward, done = env.step(action)
            total_reward += reward 
            if done: 
                next_state = None 
            memory.add_sample((state, action, reward, next_state))

            if steps > DELAY_TRAINING: 
                loss = train(primary_network, target_network, memory)
                update_network(primary_network, target_network)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)
            else: 
                loss = -1

            avg_loss += loss 
            steps += 1 

            if done: 
                if steps > DELAY_TRAINING: 
                    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
                    avg_loss /= cnt 
                    print('Episode: {}'.format(episode + 1),
                          'Total reward: {}'.format(total_reward), 
                          'Explore P: {:.4f}'.format(eps), 
                          "SOC: {:.4f}".format(env.SOC), 
                         "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
                         "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
                         )
                else: 
                    print(f"Pre-training...Episode: {episode}")
                
                episode_rewards.append(total_reward)
                episode_SOCs.append(env.SOC)
                episode_FCs.append(env.fuel_consumption)
                break 

            state = next_state 
            cnt += 1 
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }
            
    

environment version: 2
maximum steps, simulation is done ... 
Pre-training...Episode: 0
maximum steps, simulation is done ... 
Pre-training...Episode: 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer

maximum steps, simulation is done ... 
Episode: 41 Total reward: -868.0061359741658 Explore P: 0.3311 SOC: 0.7268 Cumulative_SOC_deviation: 81.3535 Fuel Consumption: 54.4714
maximum steps, simulation is done ... 
Episode: 42 Total reward: -895.1486939104486 Explore P: 0.3224 SOC: 0.7912 Cumulative_SOC_deviation: 83.5177 Fuel Consumption: 59.9715
maximum steps, simulation is done ... 
Episode: 43 Total reward: -970.0221575099534 Explore P: 0.3140 SOC: 0.8083 Cumulative_SOC_deviation: 90.8553 Fuel Consumption: 61.4696
maximum steps, simulation is done ... 
Episode: 44 Total reward: -673.3787080812323 Explore P: 0.3057 SOC: 0.7164 Cumulative_SOC_deviation: 61.9541 Fuel Consumption: 53.8379
maximum steps, simulation is done ... 
Episode: 45 Total reward: -810.300132248252 Explore P: 0.2977 SOC: 0.6737 Cumulative_SOC_deviation: 76.0117 Fuel Consumption: 50.1830
maximum steps, simulation is done ... 
Episode: 46 Total reward: -883.7883395795512 Explore P: 0.2899 SOC: 0.6323 Cumulative_SOC_de

maximum steps, simulation is done ... 
Episode: 88 Total reward: -792.5149076858864 Explore P: 0.0983 SOC: 0.7191 Cumulative_SOC_deviation: 73.8836 Fuel Consumption: 53.6793
maximum steps, simulation is done ... 
Episode: 89 Total reward: -910.798755615284 Explore P: 0.0960 SOC: 0.7309 Cumulative_SOC_deviation: 85.5658 Fuel Consumption: 55.1404
maximum steps, simulation is done ... 
Episode: 90 Total reward: -887.6031499435312 Explore P: 0.0936 SOC: 0.7696 Cumulative_SOC_deviation: 82.8993 Fuel Consumption: 58.6103
maximum steps, simulation is done ... 
Episode: 91 Total reward: -1047.5339330610614 Explore P: 0.0914 SOC: 0.8026 Cumulative_SOC_deviation: 98.6226 Fuel Consumption: 61.3079
maximum steps, simulation is done ... 
Episode: 92 Total reward: -909.7101337386171 Explore P: 0.0892 SOC: 0.7523 Cumulative_SOC_deviation: 85.2858 Fuel Consumption: 56.8518
maximum steps, simulation is done ... 
Episode: 93 Total reward: -947.8934219764167 Explore P: 0.0870 SOC: 0.7815 Cumulative_SOC_d

maximum steps, simulation is done ... 
Episode: 135 Total reward: -1422.9438071088696 Explore P: 0.0343 SOC: 0.7599 Cumulative_SOC_deviation: 136.4979 Fuel Consumption: 57.9647
maximum steps, simulation is done ... 
Episode: 136 Total reward: -442.1101218575463 Explore P: 0.0336 SOC: 0.5756 Cumulative_SOC_deviation: 40.0047 Fuel Consumption: 42.0629
maximum steps, simulation is done ... 
Episode: 137 Total reward: -2048.9374659023747 Explore P: 0.0330 SOC: 0.6096 Cumulative_SOC_deviation: 200.2396 Fuel Consumption: 46.5413
maximum steps, simulation is done ... 
Episode: 138 Total reward: -1519.1272800856373 Explore P: 0.0324 SOC: 0.8510 Cumulative_SOC_deviation: 145.4115 Fuel Consumption: 65.0120
maximum steps, simulation is done ... 
Episode: 139 Total reward: -1071.7091284976927 Explore P: 0.0318 SOC: 0.7621 Cumulative_SOC_deviation: 101.4465 Fuel Consumption: 57.2445
maximum steps, simulation is done ... 
Episode: 140 Total reward: -1937.0078724754972 Explore P: 0.0312 SOC: 0.5818 C

maximum steps, simulation is done ... 
Episode: 182 Total reward: -1350.592322833436 Explore P: 0.0167 SOC: 0.8723 Cumulative_SOC_deviation: 128.5043 Fuel Consumption: 65.5492
maximum steps, simulation is done ... 
Episode: 183 Total reward: -1112.5996782603597 Explore P: 0.0165 SOC: 0.7155 Cumulative_SOC_deviation: 105.9215 Fuel Consumption: 53.3849
maximum steps, simulation is done ... 
Episode: 184 Total reward: -2607.238477939714 Explore P: 0.0163 SOC: 0.9943 Cumulative_SOC_deviation: 253.0360 Fuel Consumption: 76.8783
maximum steps, simulation is done ... 
Episode: 185 Total reward: -1442.109766604074 Explore P: 0.0162 SOC: 0.9206 Cumulative_SOC_deviation: 137.1268 Fuel Consumption: 70.8420
maximum steps, simulation is done ... 
Episode: 186 Total reward: -720.7133936470788 Explore P: 0.0160 SOC: 0.6150 Cumulative_SOC_deviation: 67.6149 Fuel Consumption: 44.5648
maximum steps, simulation is done ... 
Episode: 187 Total reward: -2590.6555558438786 Explore P: 0.0158 SOC: 0.5340 Cumu

In [11]:
with open("DDQN3_2.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# with open("results/replay_memory_size_effect.pkl", "rb") as f: 
#     data = pickle.load(f)
    
# data